# Bài toán

> **Thu thập dữ liệu báo Việt Nam (Dantri - BTVN)**

> Hoàn thành bằng cách điền vào phần #### CODE theo hướng dẫn

    ```
    #### CODE
    ####
    ```

Cũng giống như VNExpress - Demo


Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ trang báo của Việt Nam.
- Thu thập dữ liệu (bài báo) từ các trang báo của Việt Nam để làm dữ liệu cho các bước xử lý sau.

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://dantri.com.vn/an-sinh/khong-nen-giam-che-do-om-dau-...",
        "title": "Không nên giảm chế độ ốm đau của người lao động",
        "description": "Chủ tịch công đoàn công ty Intel Product đề nghị không nên ...",
        "content": "[Giảm quyền lợi chế độ ốm đau ảnh hưởng ...",
        "metadata": {
            "cat": "AN SINH",
            "subcat": "DÂN SINH",
            "published_date": 1716159600,
            "author": "Tùng Nguyên"
        },
    },
    ```

# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [ ]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!pip install -qq chromium-chromedriver

ERROR: Could not find a version that satisfies the requirement chromium-chromedriver (from versions: none)
ERROR: No matching distribution found for chromium-chromedriver


In [2]:
# Install selenium
!pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir data

A subdirectory or file data already exists.


In [86]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json
from urllib.parse import urljoin

In [30]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [5]:
MAGAZINE_NAME = "dantri"
HOME_PAGE = "https://dantri.com.vn/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


In [6]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [7]:
# Choose the menu
all_menu = driver.find_element(by=By.CLASS_NAME, value="menu-more")
all_menu.click()

In [8]:
# Lấy hết tất cả thể loại từ đây
cats = []

row_menu = driver.find_element(by=By.CLASS_NAME, value="nf-menu")
cat_menus = row_menu.find_elements(by=By.CSS_SELECTOR, value="ol.nf-menu > li")

for cat_menu in cat_menus: # Loop through cat menus to main cat and corresponding links
    # Lấy element đầu tiên
    a_element = cat_menu.find_element(By.TAG_NAME, value="a")
    cat = a_element.text.strip()
    href = cat_menu.find_element(by=By.TAG_NAME, value="a").get_attribute("href").strip()
    # Lưu vào cats list bao gồm tên và url
    cats.append({"cat_name": cat, "url": href})

In [9]:
cats, len(cats)

([{'cat_name': 'XÃ HỘI', 'url': 'https://dantri.com.vn/xa-hoi.htm'},
  {'cat_name': 'THẾ GIỚI', 'url': 'https://dantri.com.vn/the-gioi.htm'},
  {'cat_name': 'KINH DOANH', 'url': 'https://dantri.com.vn/kinh-doanh.htm'},
  {'cat_name': 'BẤT ĐỘNG SẢN',
   'url': 'https://dantri.com.vn/bat-dong-san.htm'},
  {'cat_name': 'THỂ THAO', 'url': 'https://dantri.com.vn/the-thao.htm'},
  {'cat_name': 'VIỆC LÀM',
   'url': 'https://dantri.com.vn/lao-dong-viec-lam.htm'},
  {'cat_name': 'NHÂN ÁI', 'url': 'https://dantri.com.vn/tam-long-nhan-ai.htm'},
  {'cat_name': 'SỨC KHỎE', 'url': 'https://dantri.com.vn/suc-khoe.htm'},
  {'cat_name': 'GIẢI TRÍ', 'url': 'https://dantri.com.vn/giai-tri.htm'},
  {'cat_name': 'XE ++', 'url': 'https://dantri.com.vn/o-to-xe-may.htm'},
  {'cat_name': 'SỨC MẠNH SỐ', 'url': 'https://dantri.com.vn/suc-manh-so.htm'},
  {'cat_name': 'GIÁO DỤC', 'url': 'https://dantri.com.vn/giao-duc.htm'},
  {'cat_name': 'AN SINH', 'url': 'https://dantri.com.vn/an-sinh.htm'},
  {'cat_name': 'P

In [10]:
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt tham số

In [51]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 500 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/dantri_url.json"

# Một số thể loại không quan tâm
EXCLUDING_CATEGORIES = [] # Exclude

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

#### Chạy thử nghiệm

In [52]:
driver.get(cats[0]['url'])

In [53]:
title_news = driver.find_elements(by=By.XPATH, value="//h3[contains(@class, 'article-title') or contains(@data-seo, 'article-title')]")

In [54]:
# Lấy đường dẫn
url_new = title_news[0].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
url_new

'https://dantri.com.vn/xa-hoi/dan-no-trong-tru-so-xa-dan-quan-18-tuoi-tu-vong-20240827150451419.htm'

In [55]:
# Chỉ lấy links bắt đầu giống HOME_PAGE
url_new.startswith(HOME_PAGE)

True

In [56]:
# Tìm next page
driver.find_element(by=By.CSS_SELECTOR, value="a.page-item.next").get_attribute("href")

'https://dantri.com.vn/xa-hoi/trang-2.htm'

In [95]:
driver.close()

#### Chạy thật

In [112]:
driver = webdriver.Chrome(options=chrome_options)

In [113]:
# Global variables for filtering deduplicating urls
crawled_urls = set()

def crawl_each_category_url(driver, category_url):
    """
    Functions cho lấy urls cho từng category sau khi thử nghiệm
    """
    all_urls = set()
    url = category_url
    urls = [url]
    driver.get(url)
    btn_more = driver.find_elements(by=By.CLASS_NAME, value='btn-more')
    if btn_more:
        for btn in btn_more:
            url = btn.get_attribute("data-localpath")
            full_url = urljoin(driver.current_url, url)
            urls.append(full_url)
            
    for url in urls:
        # Limit the number of NUM_ARTICLES_PER_CAT
        while len(all_urls) < NUM_ARTICLES_PER_CAT:
            driver.get(url)
            title_news = driver.find_elements(by=By.XPATH, value="//h3[contains(@class, 'article-title') or contains(@data-seo, 'article-title')]")
            for title in title_news:
                try:
                    url_new = title.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
                    if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls: #avoid ads, different sites news
                        all_urls.add(url_new)
                        crawled_urls.add(url_new) # avoid dedup url

                # To see if there is bug
                except StaleElementReferenceException:
                    continue
                except NoSuchElementException:
                    print(f"NoSuchElementException at {url}")
                    continue
            try:
                url = driver.find_element(by=By.CSS_SELECTOR, value="a.page-item.next").get_attribute("href")
            except NoSuchElementException:
                print(f"NoSuchElementException at {url}")
                break
    
    return all_urls

In [114]:
saved_cats = {}

# Thu thập cho từng thể loại
for cat in cats:
    cat_name = cat["cat_name"]
    url = cat["url"]
    if cat_name not in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = list(urls)

with open(DATA_URL_FILE, "w", encoding="utf-8") as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

You are at {'cat_name': 'XÃ HỘI', 'url': 'https://dantri.com.vn/xa-hoi.htm'}.
You are at {'cat_name': 'THẾ GIỚI', 'url': 'https://dantri.com.vn/the-gioi.htm'}.
You are at {'cat_name': 'KINH DOANH', 'url': 'https://dantri.com.vn/kinh-doanh.htm'}.
You are at {'cat_name': 'BẤT ĐỘNG SẢN', 'url': 'https://dantri.com.vn/bat-dong-san.htm'}.
You are at {'cat_name': 'THỂ THAO', 'url': 'https://dantri.com.vn/the-thao.htm'}.
You are at {'cat_name': 'VIỆC LÀM', 'url': 'https://dantri.com.vn/lao-dong-viec-lam.htm'}.
You are at {'cat_name': 'NHÂN ÁI', 'url': 'https://dantri.com.vn/tam-long-nhan-ai.htm'}.
NoSuchElementException at https://dantri.com.vn/tam-long-nhan-ai.htm
NoSuchElementException at https://dantri.com.vn/tam-long-nhan-ai/nhip-cau-nhan-ai/trang-17.htm
NoSuchElementException at https://dantri.com.vn/tam-long-nhan-ai/vuot-len-so-phan/trang-3.htm
You are at {'cat_name': 'SỨC KHỎE', 'url': 'https://dantri.com.vn/suc-khoe.htm'}.
You are at {'cat_name': 'GIẢI TRÍ', 'url': 'https://dantri.com

In [115]:
len(crawled_urls)

9101

In [116]:
for i in saved_cats:
    print(i, len(saved_cats[i]))

XÃ HỘI 520
THẾ GIỚI 503
KINH DOANH 500
BẤT ĐỘNG SẢN 526
THỂ THAO 510
VIỆC LÀM 522
NHÂN ÁI 242
SỨC KHỎE 541
GIẢI TRÍ 524
XE ++ 530
SỨC MẠNH SỐ 530
GIÁO DỤC 525
AN SINH 520
PHÁP LUẬT 528
DU LỊCH 503
ĐỜI SỐNG 521
TÌNH YÊU 528
KHOA HỌC - CÔNG NGHỆ 528


### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [143]:
# Filepath cho cái trước
FILE_URL_PATH = "data/dantri_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = None # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data/dantri"
#
os.makedirs(DATA_FOLDER_OUTPUT, exist_ok=True)

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

In [144]:
# Đọc file url
with open(FILE_URL_PATH, "r", encoding='utf-8') as fIn:
    url_data = json.load(fIn)

len(url_data)

18

#### Chạy thử nghiệm

In [145]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm
SAMPLE_ARTICLE_URLS = [
    "https://dantri.com.vn/xa-hoi/hang-van-du-khach-o-lai-tren-cac-dao-quang-ninh-hai-phong-cho-bao-tan-20240722185725192.htm",
    "https://dantri.com.vn/xa-hoi/nhieu-du-khach-ngoi-thung-o-to-ban-tai-luu-thong-tren-duong-pho-sam-son-20240722163805286.htm",
    "https://dantri.com.vn/the-gioi/ong-trump-doi-boi-thuong-sau-khi-tong-thong-biden-dung-tranh-cu-20240722215254170.htm",
    "https://dantri.com.vn/lao-dong-viec-lam/lao-dong-viet-ve-chan-dung-bac-ho-tong-bi-thu-nguyen-phu-trong-bang-2-tay-20240721101224503.htm",
    "https://dantri.com.vn/khoa-hoc-cong-nghe/chu-cho-lac-nha-dung-dam-mua-nhieu-ngay-cho-chu-den-don-20240722153845731.htm",
    "https://dantri.com.vn/giai-tri/muon-kiep-nhan-sinh-cua-gs-john-vu-nguyen-phong-sap-phat-hanh-20200430134253084.htm" # Author
    # "https://dantri.com.vn/xa-hoi/nguoi-dan-tphcm-xep-hang-thap-huong-tuong-nho-tong-bi-thu-nguyen-phu-trong-20240722193718325.htm" # we ignore this one (emagazine), maybe extensions in the future if you want handle indepedently,
    # "https://dantri.com.vn/xa-hoi/toan-van-bai-viet-khoi-day-niem-tu-hao-ve-dang-cua-tong-bi-thu-20240131111512837.htm" # similar reason like above
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[-3]

In [146]:
driver.get(SAMPLE_ARTICLE_URL)

In [147]:
# We ignore emagazine for now ..

is_emagazine = False
try:
    driver.find_element(by=By.CLASS_NAME, value="emagazine")
except NoSuchElementException:
    is_emagazine = True

is_emagazine

True

In [1]:
# Tìm kiếm title

driver.find_element(by=By.CSS_SELECTOR, value="h1.title-page.detail").text

NameError: name 'driver' is not defined

In [149]:
# Tìm kiếm description

description = driver.find_element(by=By.CLASS_NAME, value="singular-sapo").text

# Bỏ (dân trí) ở đầu
DANTRI_PHRASE = "(Dân trí) - "
if DANTRI_PHRASE in description:
    description = description.replace(DANTRI_PHRASE, "")

description

'Đoạn clip du học sinh Việt ở Nhật thể hiện chân dung Chủ tịch Hồ Chí Minh và Tổng Bí thư Nguyễn Phú Trọng với 2 tay cầm bút song song khiến nhiều người ngạc nhiên, xúc động.'

In [150]:
# Thu thập thể loại
lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.dt-text-c808080").find_elements(by=By.TAG_NAME, value="li")
main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None
main_cat, sub_cat

('LAO ĐỘNG - VIỆC LÀM', 'NHÂN LỰC MỚI')

In [151]:
# Thu thập ngày
publish_date = driver.find_element(by=By.CSS_SELECTOR, value='time.author-time').get_attribute("datetime").strip()
publish_date

'2024-07-22 05:59'

In [152]:
# Thu thập tác giả

author = "Unknown"
try:
    author = driver.find_element(by=By.CSS_SELECTOR, value="div.author-name > a").text
except:
    pass

author

'Sơn Nguyễn'

In [153]:
# Tìm kiếm contents
article = driver.find_element(by=By.CSS_SELECTOR, value="div.singular-content")
children = article.find_elements(by=By.XPATH, value="./*")

In [154]:
# Thu thập contents
contents = []

for idx, child in enumerate(children):
    text = child.text.strip()
    # right align
    if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph
        # If center
        if len(text):
            if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        if len(text):
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "table": # Do nothing rightnow
        pass

In [155]:
contents

['Hoàng Văn Vững (23 tuổi, quê Nghệ An), du học sinh đang sinh sống tại tỉnh Hiroshima (Nhật Bản), khiến nhiều người rơi nước mắt với đoạn clip ghi lại quá trình vẽ chân dung Bác Hồ và Tổng Bí thư Nguyễn Phú Trọng.',
 'Chia sẻ với phóng viên Dân trí, Vững cho biết, anh cũng như cộng đồng người Việt Nam hiện sinh sống, làm việc ở nước ngoài rất buồn, tiếc thương khi nghe tin Tổng Bí Thư Nguyễn Phú Trọng từ trần.',
 '[Phát Video\nChàng trai vẽ chân dung Bác Hồ và Tổng Bí thư Nguyễn Phú Trọng bằng hai tay (Clip: NVCC).]',
 '"Không riêng gì tôi mà người dân Việt Nam trong và ngoài nước rất tiếc thương và muốn tự mình làm điều gì đó để thể hiện tình cảm đối với Tổng Bí thư Nguyễn Phú Trọng.',
 'Mọi người để ảnh không màu hoặc treo cờ rủ. Với tôi, khi nghe tin đau buồn này, tôi chọn cách ngồi vẽ. Tôi quyết định tái hiện chân dung hai người con đặc biệt của dân tộc Việt Nam, Bác Hồ và Tổng Bí thư Nguyễn Phú Trọng trong cùng một bức tranh, vẽ cùng lúc bằng 2 tay.',
 'Tôi mất hơn 4 tiếng để hoà

In [156]:
driver.close()

#### Chạy thật

In [157]:

# Class Exception for ignore emagazine
class MagazineArticleException(Exception):
    pass

def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)


    # Tạm thời bỏ qua emagazine
    is_emagazine = True
    try:
        driver.find_element(by=By.CLASS_NAME, value="emagazine")
    except NoSuchElementException:
        is_emagazine = False

    if is_emagazine:
        raise MagazineArticleException("We ignore MagazineArticle")

    # Thu thập title
    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.title-page.detail").text

    # Thu thập description
    description = driver.find_element(by=By.CLASS_NAME, value="singular-sapo").text

    # Bỏ (dân trí) ở đầu
    DANTRI_PHRASE = "(Dân trí) - "
    if DANTRI_PHRASE in description:
        description = description.replace(DANTRI_PHRASE, "")

    # Thu thập thể loại
    lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.dt-text-c808080").find_elements(by=By.TAG_NAME, value="li")
    main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
    sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None

    # Thu thập published date
    publish_date = driver.find_element(by=By.CSS_SELECTOR, value='time.author-time').get_attribute("datetime").strip()

    # Thu thập người viết
    author = "Unknown"
    try:
        author = driver.find_element(by=By.CSS_SELECTOR, value="div.author-name > a").text
    except:
        pass

    # Thu thập content bài báo
    article = driver.find_element(by=By.CSS_SELECTOR, value="div.singular-content")
    children = article.find_elements(by=By.XPATH, value="./*")
    contents = []

    for idx, child in enumerate(children):
        text = child.text.strip()
        # right align
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph
            # If center
            if len(text):
                if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)
        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if len(text):
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)
        elif child.tag_name == "table": # Do nothing rightnow
            pass

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": publish_date,
            "author": author
        }
    }


In [158]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        # NoSuchElementException
        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue

        except MagazineArticleException as e:
            continue


    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding='utf-8') as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

Thu thập dữ liệu thể loại XÃ HỘI ..
Thu thập dữ liệu thể loại THẾ GIỚI ..
Thu thập dữ liệu thể loại KINH DOANH ..
Bug at url: https://dantri.com.vn/kinh-doanh/chuyen-gia-blockchain-dinh-quang-loc-viet-nam-la-thi-truong-day-thu-vi-de-khai-pha-20211030062710718.htm, with NoSuchElementException
Thu thập dữ liệu thể loại BẤT ĐỘNG SẢN ..
Thu thập dữ liệu thể loại THỂ THAO ..
Bug at url: https://dantri.com.vn/the-thao/jannik-sinner-khong-bi-cam-thi-dau-du-duong-tinh-voi-doping-20240821064459506.htm, with NoSuchElementException
Bug at url: https://dantri.com.vn/the-thao/djokovic-duoc-du-doan-giai-nghe-neu-vo-dich-us-open-2024-20240824080505228.htm, with NoSuchElementException
Bug at url: https://dantri.com.vn/the-thao/sinner-lan-dau-vo-dich-cincinnati-open-20240820072719553.htm, with NoSuchElementException
Bug at url: https://dantri.com.vn/the-thao/djokovic-du-giai-pickleball-truoc-them-us-open-2024-20240820081241428.htm, with NoSuchElementException
Bug at url: https://dantri.com.vn/the-thao/

In [159]:
# Xem 1 sample
cat_data[-1]

{'url': 'https://dantri.com.vn/khoa-hoc-cong-nghe/chuot-tui-xuat-hien-o-cao-bang-co-phai-kangaroo-20231110145244423.htm',
 'title': 'Chuột túi xuất hiện ở Cao Bằng có phải Kangaroo?',
 'description': 'Hình ảnh một sinh vật giống chuột túi chạy rông dọc một tuyến đường ở tỉnh Cao Bằng khiến cư dân mạng xôn xao, bởi lẽ trong tự nhiên, đây là loài vật không phân bố tại Việt Nam.',
 'content': 'Chiều 10/11, mạng xã hội tại Việt Nam lan truyền một đoạn clip ghi lại hình ảnh sinh vật giống chuột túi, di chuyển dọc theo một tuyến đường thuộc địa phận huyện Thạch An (tỉnh Cao Bằng).\nTrao đổi với phóng viên Dân trí, ông Hoàng Văn Khanh, Chủ tịch UBND xã Đức Long, xác nhận thông tin trên và cho biết, con chuột túi đầu tiên được người dân phát hiện và bắt giữ tối 8/11.\n[Phát Video\nSinh vật giống chuột túi xuất hiện tại Cao Bằng gây xôn xao (Video: Facebook).]\nĐoạn clip đã "gây sốt" mạng xã hội và khiến cư dân mạng xôn xao, bởi lẽ chuột túi là loài động vật bản địa của Úc, nên việc con vật này

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB